## 一、为什么需要LSTM？

### SimpleRNN的局限性

传统RNN在处理长序列时存在以下问题：

1. **梯度消失(Vanishing Gradient)**
   - 梯度在反向传播时呈指数级衰减
   - 网络难以学习长期依赖关系
   - 只能记住最近几个时间步的信息

2. **梯度爆炸(Exploding Gradient)**
   - 梯度可能呈指数级增长
   - 导致权重更新不稳定
   - 训练过程难以收敛

### LSTM的解决方案

LSTM通过引入**门控机制(Gating Mechanism)**和**细胞状态(Cell State)**来解决这些问题：

- **细胞状态**：作为信息高速公路，可以让信息不经过非线性变换直接传递
- **三个门**：控制信息的流动，决定保留、遗忘或输出哪些信息

## 二、LSTM的结构详解

### LSTM的核心组件

LSTM单元包含以下关键组件：

#### 1. 遗忘门(Forget Gate)
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

作用：决定从细胞状态中丢弃哪些信息
- 输出范围：[0, 1]
- 0 = 完全遗忘，1 = 完全保留

#### 2. 输入门(Input Gate)
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
$$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

作用：决定向细胞状态中添加哪些新信息

#### 3. 细胞状态更新
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$

作用：结合遗忘门和输入门的结果，更新细胞状态

#### 4. 输出门(Output Gate)
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \odot \tanh(C_t)$$

作用：决定输出细胞状态的哪些部分

### 符号说明
- $\sigma$: sigmoid函数
- $\odot$: 逐元素乘法(Hadamard product)
- $[h_{t-1}, x_t]$: 拼接上一时刻隐藏状态和当前输入

In [ ]:
"""
三、Keras中的LSTM实现
演示不同的LSTM构建方式
"""
from tensorflow import keras
import numpy as np

print("=" * 70)
print("方法1: 使用Sequential API构建LSTM模型")
print("=" * 70)

# 构建基础LSTM模型
model_simple = keras.models.Sequential([
    # 显式指定输入形状
    keras.Input(shape=[None, 1]),  # [timesteps, features]
    
    # LSTM层
    keras.layers.LSTM(50, return_sequences=True),
    keras.layers.LSTM(50),
    keras.layers.Dense(1)
])

model_simple.summary()
print(f"\n总参数量: {model_simple.count_params():,}")

print("\n" + "=" * 70)
print("方法2: 使用LSTMCell构建自定义循环")
print("=" * 70)

# 使用LSTMCell提供更细粒度的控制
model_cell = keras.models.Sequential([
    keras.layers.RNN(
        keras.layers.LSTMCell(20),
        return_sequences=True,
        input_shape=[None, 1]
    ),
    keras.layers.RNN(
        keras.layers.LSTMCell(20),
        return_sequences=True
    ),
    # TimeDistributed: 对每个时间步应用相同的Dense层
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model_cell.summary()
print(f"\n总参数量: {model_cell.count_params():,}")

print("\n" + "=" * 70)
print("LSTM参数计算公式")
print("=" * 70)
print("LSTM单元有4组权重矩阵(遗忘门、输入门、输出门、候选值)")
print("参数量 = 4 × (units × (units + input_dim + 1))")
print("\n示例：units=50, input_dim=1")
print(f"参数量 = 4 × (50 × (50 + 1 + 1)) = {4 * (50 * (50 + 1 + 1)):,}")

## 四、实战：LSTM用于时间序列预测

我们将创建一个合成的正弦波数据集，并使用LSTM进行预测。这是一个经典的时间序列任务，可以很好地展示LSTM的能力。

In [ ]:
"""
生成时间序列数据
创建正弦波叠加噪声的数据集
"""
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
np.random.seed(42)

def generate_time_series(batch_size, n_steps):
    """
    生成多个时间序列样本
    
    参数:
        batch_size: 生成的样本数量
        n_steps: 每个序列的时间步数
    
    返回:
        形状为 (batch_size, n_steps, 1) 的数组
    """
    # 生成基础频率
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    
    # 生成时间点
    time = np.linspace(0, 1, n_steps)
    
    # 生成正弦波叠加
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  # 第一个正弦波
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # 第二个正弦波
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # 添加噪声
    
    return series[..., np.newaxis].astype(np.float32)

# 生成数据集
n_steps = 50
series = generate_time_series(10000, n_steps + 1)

# 划分输入和目标
# 使用前n_steps个点预测第n_steps+1个点
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

print("=" * 70)
print("数据集信息")
print("=" * 70)
print(f"训练集形状: X={X_train.shape}, y={y_train.shape}")
print(f"验证集形状: X={X_valid.shape}, y={y_valid.shape}")
print(f"测试集形状: X={X_test.shape}, y={y_test.shape}")

# 可视化几个样本
fig, axes = plt.subplots(2, 3, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    ax.plot(X_train[i, :, 0], 'b-', linewidth=2, label='输入序列')
    ax.axhline(y=y_train[i, 0], color='r', linestyle='--', linewidth=2, label='目标值')
    ax.set_title(f'样本 {i+1}', fontsize=12, fontweight='bold')
    ax.set_xlabel('时间步', fontsize=10)
    ax.set_ylabel('值', fontsize=10)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n数据生成完成！")

In [ ]:
"""
构建并训练LSTM模型
使用SimpleRNN和LSTM进行对比
"""
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

print("=" * 70)
print("构建并训练SimpleRNN模型（作为基准）")
print("=" * 70)

# 构建SimpleRNN模型
model_rnn = Sequential([
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20),
    Dense(1)
])

model_rnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# 训练SimpleRNN（快速测试，少量epoch）
history_rnn = model_rnn.fit(
    X_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=1
)

print("\n" + "=" * 70)
print("构建并训练LSTM模型")
print("=" * 70)

# 构建LSTM模型
model_lstm = Sequential([
    LSTM(20, return_sequences=True, input_shape=[None, 1]),
    LSTM(20),
    Dense(1)
])

model_lstm.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# 训练LSTM
history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=5,                    # 测试时使用较少epoch
    batch_size=32,
    validation_data=(X_valid, y_valid),
    verbose=1
)

print("\n" + "=" * 70)
print("模型性能对比")
print("=" * 70)

# 在测试集上评估
rnn_loss, rnn_mae = model_rnn.evaluate(X_test, y_test, verbose=0)
lstm_loss, lstm_mae = model_lstm.evaluate(X_test, y_test, verbose=0)

print(f"\nSimpleRNN - 测试集MSE: {rnn_loss:.6f}, MAE: {rnn_mae:.6f}")
print(f"LSTM      - 测试集MSE: {lstm_loss:.6f}, MAE: {lstm_mae:.6f}")
print(f"\n性能提升: {((rnn_mae - lstm_mae) / rnn_mae * 100):.2f}%")

In [ ]:
"""
可视化预测结果
对比SimpleRNN和LSTM的预测效果
"""
import matplotlib.pyplot as plt

# 在测试集上进行预测
y_pred_rnn = model_rnn.predict(X_test, verbose=0)
y_pred_lstm = model_lstm.predict(X_test, verbose=0)

# 选择几个样本进行可视化
n_samples = 3
fig, axes = plt.subplots(n_samples, 1, figsize=(14, 4*n_samples))

for i in range(n_samples):
    ax = axes[i] if n_samples > 1 else axes
    
    # 绘制输入序列
    ax.plot(range(n_steps), X_test[i, :, 0], 'b-', 
            linewidth=2, label='输入序列', alpha=0.7)
    
    # 绘制真实值
    ax.plot(n_steps, y_test[i, 0], 'go', 
            markersize=12, label='真实值')
    
    # 绘制SimpleRNN预测
    ax.plot(n_steps, y_pred_rnn[i, 0], 'rs', 
            markersize=12, label=f'SimpleRNN预测')
    
    # 绘制LSTM预测
    ax.plot(n_steps, y_pred_lstm[i, 0], 'md', 
            markersize=12, label=f'LSTM预测')
    
    ax.set_title(f'测试样本 {i+1}', fontsize=13, fontweight='bold')
    ax.set_xlabel('时间步', fontsize=11)
    ax.set_ylabel('值', fontsize=11)
    ax.legend(fontsize=10, loc='best')
    ax.grid(True, alpha=0.3)
    
    # 添加误差标注
    rnn_error = abs(y_test[i, 0] - y_pred_rnn[i, 0])
    lstm_error = abs(y_test[i, 0] - y_pred_lstm[i, 0])
    ax.text(0.02, 0.98, 
            f'SimpleRNN误差: {rnn_error:.4f}\nLSTM误差: {lstm_error:.4f}',
            transform=ax.transAxes,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontsize=10)

plt.tight_layout()
plt.show()

# 绘制训练历史对比
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 损失曲线
epochs_range = range(1, len(history_rnn.history['loss']) + 1)
ax1.plot(epochs_range, history_rnn.history['loss'], 'b-o', 
         label='SimpleRNN训练', linewidth=2)
ax1.plot(epochs_range, history_rnn.history['val_loss'], 'b--o', 
         label='SimpleRNN验证', linewidth=2)
ax1.plot(epochs_range, history_lstm.history['loss'], 'r-s', 
         label='LSTM训练', linewidth=2)
ax1.plot(epochs_range, history_lstm.history['val_loss'], 'r--s', 
         label='LSTM验证', linewidth=2)
ax1.set_title('训练损失对比', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('MSE Loss', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# MAE曲线
ax2.plot(epochs_range, history_rnn.history['mae'], 'b-o', 
         label='SimpleRNN训练', linewidth=2)
ax2.plot(epochs_range, history_rnn.history['val_mae'], 'b--o', 
         label='SimpleRNN验证', linewidth=2)
ax2.plot(epochs_range, history_lstm.history['mae'], 'r-s', 
         label='LSTM训练', linewidth=2)
ax2.plot(epochs_range, history_lstm.history['val_mae'], 'r--s', 
         label='LSTM验证', linewidth=2)
ax2.set_title('MAE对比', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('MAE', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("结论")
print("=" * 70)
print("LSTM通过其门控机制能够:")
print("1. 更好地捕捉长期依赖关系")
print("2. 减少梯度消失问题")
print("3. 在时间序列预测任务上通常优于SimpleRNN")
print("=" * 70)